In [1]:
import pandas as pd
import numpy as np

## Clean up building data

Start with the raw 2019 data from the City of Seattle.

Calculate extra fields:
- OSE Building Type: mapping between the City's Building Type and the building types used by OSE. See `city_building_types.csv` for mapping. These are generic categories like "Nonresidential" and "Multifamily".
- OSE Property Use fields: this is a mapping of EPA property use types to the types used by OSE. See `building_activity_types.csv` for mapping. These are more detailed categories like "Hospital" or "Restaurant".
- Property Use Type GFA for Policy fields: the GFA for Policy is the same as the GFA unless it is one of the use types "Data Center" and "Parking" that aren't subject to the policy (then it's 0).
- Total GFA for Policy: sum of the Property Use Type GFA for Policy fields
- PropertyUseType Percent GFA fields: percent of a building's total GFA that is from the given property use type. Zero if the use type is not covered by the policy

In [2]:
raw_building_data = pd.read_csv('seattle_large_building_data_2019.csv')

In [3]:
raw_building_data.columns

Index(['OSEBuildingID', 'TaxParcelIdentificationNumber', 'DataYear',
       'BuildingType', 'BuildingName', 'CouncilDistrictCode', 'Neighborhood',
       'YearBuilt', 'Address', 'City', 'State', 'ZipCode', 'Latitude',
       'Longitude', 'PrimaryPropertyType', 'ListOfAllPropertyUseTypes',
       'EPAPropertyType', 'TotalGHGEmissions', 'ENERGYSTARScore',
       'YearsENERGYSTARCertified', 'SiteEUI(kBtu/sf)', 'SiteEUIWN(kBtu/sf)',
       'SiteEnergyUse(kBtu)', 'SiteEnergyUseWN(kBtu)', 'SourceEUI(kBtu/sf)',
       'SourceEUIWN(kBtu/sf)', 'GHGEmissions(MetricTonsCO2e)',
       'GHGEmissionsIntensity', 'GHGEmissionsIntensity(kgCO2e/ft2)',
       'NaturalGas(kBtu)', 'NaturalGas(therms)', 'Electricity(kBtu)',
       'Electricity(kWh)', 'SteamUse(kBtu)', 'OtherFuelUse(kBtu)',
       'PropertyGFATotal', 'PropertyGFABuilding(s)', 'PropertyGFAParking',
       'NumberofBuildings', 'NumberofFloors', 'LargestPropertyUseType',
       'LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseType',
      

In [4]:
cleaned_building_data = raw_building_data[[
    'OSEBuildingID', 
    'TaxParcelIdentificationNumber', 
    'DataYear',
    'BuildingType', 
    'BuildingName', 
    'NaturalGas(kBtu)', 
    'Electricity(kBtu)', 
    'SteamUse(kBtu)', 
    'PropertyGFATotal', 
    'PropertyGFABuilding(s)', 
    'PropertyGFAParking', 
    'LargestPropertyUseType',
    'LargestPropertyUseTypeGFA', 
    'SecondLargestPropertyUseType',
    'SecondLargestPropertyUseTypeGFA', 
    'ThirdLargestPropertyUseType',
    'ThirdLargestPropertyUseTypeGFA'
]]

In [5]:
# Map OSE Building Type

city_building_types_mapping = pd.read_csv('city_building_types.csv')
building_types = dict(zip(list(city_building_types_mapping['BuildingType (City classification)']), list(city_building_types_mapping['Type (Legislative classification)'])))

In [6]:
building_types

{'NonResidential': 'NonResidential',
 'Nonresidential COS': 'NonResidential',
 'Nonresidential WA': 'NonResidential',
 'Multifamily LR (1-4)': 'Multifamily',
 'Multifamily MR (5-9)': 'Multifamily',
 'Multifamily HR (10+)': 'Multifamily',
 'SPS-District K-12': 'Campus',
 'Campus': 'Campus'}

In [7]:
cleaned_building_data['OSE Building Type'] = cleaned_building_data.apply(lambda building: building_types[building['BuildingType']], axis=1)

/var/folders/dv/sn1_d0xn0vncjbncdpkkg_pc0000gn/T/ipykernel_10068/356869480.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_building_data['OSE Building Type'] = cleaned_building_data.apply(lambda building: building_types[building['BuildingType']], axis=1)


In [8]:
cleaned_building_data[['BuildingType', 'OSE Building Type']].sample(10)

,BuildingType,OSE Building Type
3413,NonResidential,NonResidential
1509,Multifamily MR (5-9),Multifamily
830,Multifamily LR (1-4),Multifamily
1959,Multifamily LR (1-4),Multifamily
1801,Multifamily LR (1-4),Multifamily
5,Nonresidential COS,NonResidential
2483,Multifamily LR (1-4),Multifamily
2766,Multifamily MR (5-9),Multifamily
3293,Multifamily MR (5-9),Multifamily
3434,Multifamily LR (1-4),Multifamily


In [9]:
# Map use types

In [10]:
ose_use_types = pd.read_csv('building_activity_types.csv')
ose_use_types.columns
ose_use_types_mapping = dict(zip(ose_use_types['EPA Building Type'], ose_use_types['OSE Building Type']))

In [11]:
ose_use_types_mapping

{'Hotel': 'Hotel',
 'Police Station': 'Fire/Police Station',
 'Other - Entertainment/Public Assembly': 'Entertainment/Public Assembly',
 'Multifamily Housing': 'Multifamily Housing',
 'Library': 'Services',
 'Fitness Center/Health Club/Gym': 'Recreation',
 'Social/Meeting Hall': 'Entertainment/Public Assembly',
 'Courthouse': 'Other',
 'Prison/Incarceration': 'Other',
 'K-12 School': 'K-12 School',
 'College/University': 'College/University',
 'Office': 'Office',
 'Self-Storage Facility': 'Self-Storage Facility',
 'Other - Mall': 'Retail Store',
 'Senior Care Community': 'Senior Living Community',
 'Medical Office': 'Office',
 'Other': 'Other',
 'Performing Arts': 'Entertainment/Public Assembly',
 'Supermarket/Grocery Store': 'Supermarket/Grocery Store',
 'Hospital (General Medical & Surgical)': 'Hospital',
 'Fire Station': 'Fire/Police Station',
 'Museum': 'Entertainment/Public Assembly',
 'Repair Services (Vehicle, Shoe, Locksmith, etc)': 'Services',
 'Other - Lodging/Residential': '

In [12]:
cleaned_building_data['LargestPropertyUseType OSE'] = cleaned_building_data.apply(lambda building: ose_use_types_mapping[building['LargestPropertyUseType']], axis=1)
cleaned_building_data['SecondLargestPropertyUseType OSE'] = cleaned_building_data.apply(lambda building: ose_use_types_mapping[building['SecondLargestPropertyUseType']], axis=1)
cleaned_building_data['ThirdLargestPropertyUseType OSE'] = cleaned_building_data.apply(lambda building: ose_use_types_mapping[building['ThirdLargestPropertyUseType']], axis=1)

/var/folders/dv/sn1_d0xn0vncjbncdpkkg_pc0000gn/T/ipykernel_10068/4016502074.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_building_data['LargestPropertyUseType OSE'] = cleaned_building_data.apply(lambda building: ose_use_types_mapping[building['LargestPropertyUseType']], axis=1)
/var/folders/dv/sn1_d0xn0vncjbncdpkkg_pc0000gn/T/ipykernel_10068/4016502074.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_building_data['SecondLargestPropertyUseType OSE'] = cleaned_building_data.apply(la

In [13]:
cleaned_building_data[['LargestPropertyUseType', 'LargestPropertyUseType OSE']].sample(10)

,LargestPropertyUseType,LargestPropertyUseType OSE
3462,Multifamily Housing,Multifamily Housing
1784,Multifamily Housing,Multifamily Housing
1199,Multifamily Housing,Multifamily Housing
1705,Multifamily Housing,Multifamily Housing
3435,Multifamily Housing,Multifamily Housing
2740,Multifamily Housing,Multifamily Housing
2060,Office,Office
925,Multifamily Housing,Multifamily Housing
2931,Multifamily Housing,Multifamily Housing
1175,"Repair Services (Vehicle, Shoe, Locksmith, etc)",Services


In [14]:
cleaned_building_data[['SecondLargestPropertyUseType', 'SecondLargestPropertyUseType OSE']].sample(10)

,SecondLargestPropertyUseType,SecondLargestPropertyUseType OSE
1364,NaN,NaN
2681,Parking,NaN
1363,NaN,NaN
1498,Parking,NaN
3498,NaN,NaN
3504,NaN,NaN
1694,NaN,NaN
2011,Parking,NaN
958,Parking,NaN
2992,NaN,NaN


In [15]:
cleaned_building_data[['ThirdLargestPropertyUseType', 'ThirdLargestPropertyUseType OSE']].sample(10)

,ThirdLargestPropertyUseType,ThirdLargestPropertyUseType OSE
3102,Parking,NaN
3142,NaN,NaN
1145,NaN,NaN
0,NaN,NaN
877,Parking,NaN
2218,NaN,NaN
2750,NaN,NaN
1873,NaN,NaN
733,NaN,NaN
1350,NaN,NaN


In [16]:
cleaned_building_data['LargestPropertyUseType OSE'].unique()

array(['Hotel', 'Fire/Police Station', 'Entertainment/Public Assembly',
       'Multifamily Housing', 'Services', 'Recreation', 'Other',
       'K-12 School', 'College/University', 'Office',
       'Self-Storage Facility', 'Retail Store', 'Senior Living Community',
       'Supermarket/Grocery Store', 'Hospital',
       'Residence Hall/Dormitory', 'Non-Refrigerated Warehouse', nan,
       'Worship Facility', 'Laboratory', 'Restaurant',
       'Refrigerated Warehouse'], dtype=object)

In [17]:
cleaned_building_data[cleaned_building_data['LargestPropertyUseType OSE'].isnull()]['LargestPropertyUseType']

158         Parking
233     Data Center
239         Parking
247             NaN
283         Parking
           ...     
3524            NaN
3526            NaN
3527            NaN
3529            NaN
3532            NaN
Name: LargestPropertyUseType, Length: 99, dtype: object

In [18]:
# some buildings don't have a largest EPA use type at all
# we can't do anything about these buildings, so we'll drop them

cleaned_building_data = cleaned_building_data[~cleaned_building_data['LargestPropertyUseType'].isnull()]

In [27]:
# Property Use Type GFA for Policy

def get_property_use_type_gfa_for_policy(building, largest_property_type_ose, largest_property_type_gfa):
    return 0 if building[largest_property_type_] in ['Data Center', 'Parking', np.nan] else building[largest_property_type_gfa]

cleaned_building_data['LargestPropertyUseType OSE GFA for Policy'] = cleaned_building_data.apply(lambda building: get_property_use_type_gfa_for_policy(building, 'LargestPropertyUseType OSE', 'LargestPropertyUseTypeGFA'), axis=1)
cleaned_building_data['SecondLargestPropertyUseType OSE GFA for Policy'] = cleaned_building_data.apply(lambda building: get_property_use_type_gfa_for_policy(building, 'SecondLargestPropertyUseType OSE', 'SecondLargestPropertyUseTypeGFA'), axis=1)
cleaned_building_data['ThirdLargestPropertyUseType OSE GFA for Policy'] = cleaned_building_data.apply(lambda building: get_property_use_type_gfa_for_policy(building, 'ThirdLargestPropertyUseType OSE', 'ThirdLargestPropertyUseTypeGFA'), axis=1)

In [28]:
cleaned_building_data[['LargestPropertyUseType OSE GFA for Policy', 'SecondLargestPropertyUseType OSE GFA for Policy', 'ThirdLargestPropertyUseType OSE GFA for Policy']].head()

,LargestPropertyUseType OSE GFA for Policy,SecondLargestPropertyUseType OSE GFA for Policy,ThirdLargestPropertyUseType OSE GFA for Policy
0,88434.0,0.0,0.0
1,83880.0,0.0,4622.0
2,756493.0,0.0,0.0
3,61320.0,0.0,0.0
4,123445.0,0.0,0.0


In [29]:
# replace NaN with 0
cleaned_building_data[['LargestPropertyUseType OSE GFA for Policy', 'SecondLargestPropertyUseType OSE GFA for Policy', 'ThirdLargestPropertyUseType OSE GFA for Policy']] = cleaned_building_data[['LargestPropertyUseType OSE GFA for Policy', 'SecondLargestPropertyUseType OSE GFA for Policy', 'ThirdLargestPropertyUseType OSE GFA for Policy']].fillna(0)

In [30]:
cleaned_building_data['Total GFA for Policy'] = cleaned_building_data.apply(lambda building: building['LargestPropertyUseType OSE GFA for Policy'] + building['SecondLargestPropertyUseType OSE GFA for Policy'] + building['ThirdLargestPropertyUseType OSE GFA for Policy'], axis=1)

In [31]:
cleaned_building_data[cleaned_building_data['Total GFA for Policy'] == 0]

,OSEBuildingID,TaxParcelIdentificationNumber,DataYear,BuildingType,BuildingName,NaturalGas(kBtu),Electricity(kBtu),SteamUse(kBtu),PropertyGFATotal,PropertyGFABuilding(s),...,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,OSE Building Type,LargestPropertyUseType OSE,SecondLargestPropertyUseType OSE,ThirdLargestPropertyUseType OSE,LargestPropertyUseType OSE GFA for Policy,SecondLargestPropertyUseType OSE GFA for Policy,ThirdLargestPropertyUseType OSE GFA for Policy,Total GFA for Policy
233,365,1142000290,2019,NonResidential,UW TOWER DATA CENTER,0,33303297,0,36000,36000,...,NaN,NaN,NonResidential,NaN,NaN,NaN,0.0,0.0,0.0,0.0


In [33]:
# Calculate percent for each use type
cleaned_building_data['LargestPropertyUseType Percent GFA'] = cleaned_building_data.apply(lambda building: building['LargestPropertyUseType OSE GFA for Policy'] / building['Total GFA for Policy'] if building['Total GFA for Policy'] > 0 else 0, axis=1)
cleaned_building_data['SecondLargestPropertyUseType Percent GFA'] = cleaned_building_data.apply(lambda building: building['SecondLargestPropertyUseType OSE GFA for Policy'] / building['Total GFA for Policy'] if building['Total GFA for Policy'] > 0 else 0, axis=1)
cleaned_building_data['ThirdLargestPropertyUseType Percent GFA'] = cleaned_building_data.apply(lambda building: building['ThirdLargestPropertyUseType OSE GFA for Policy'] / building['Total GFA for Policy'] if building['Total GFA for Policy'] > 0 else 0, axis=1)

In [34]:
# Add size classification

def classify_size(sq_ft):
    """
    Use letter classifications for building size instead of dealing with size ranges (>220k, 90-220k, etc.)
    """
    if sq_ft > 220000:
        return 'A'
    elif sq_ft > 90000:
        return 'B'
    elif sq_ft > 50000:
        return 'C'
    elif sq_ft > 30000:
        return 'D'
    elif sq_ft > 20000:
        return 'E'
    else:
        return 'F'
        
cleaned_building_data['sq_ft_classification'] = cleaned_building_data['Total GFA for Policy'].apply(lambda building: classify_size(building))

In [35]:
cleaned_building_data.to_csv('cleaned_building_data_with_policy_gfa.csv')